<a href="https://colab.research.google.com/github/microsoft/autogen/blob/main/notebook/agentchat_groupchat_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Auto Generated Agent Chat: Group Chat with Retrieval Augmented Generation

AutoGen supports conversable agents powered by LLMs, tools, or humans, performing tasks collectively via automated chat. This framework allows tool use and human participation through multi-agent conversation.
Please find documentation about this feature [here](https://microsoft.github.io/autogen/docs/Use-Cases/agent_chat).

## Requirements

AutoGen requires `Python>=3.8`. To run this notebook example, please install:
```bash
pip install "pyautogen[retrievechat]>=0.2.3"
```

In [1]:
%%capture --no-stderr
# %pip install "pyautogen[retrievechat]>=0.2.3"

## Set your API Endpoint

The [`config_list_from_json`](https://microsoft.github.io/autogen/docs/reference/oai/openai_utils#config_list_from_json) function loads a list of configurations from an environment variable or a json file.

In [2]:
import chromadb

import autogen
from autogen import AssistantAgent
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    file_location=".",
    filter_dict={
        "model": ["gpt-3.5-turbo", "gpt-35-turbo", "gpt-35-turbo-0613", "gpt-4", "gpt4", "gpt-4-32k"],
    },
)

print("LLM models: ", [config_list[i]["model"] for i in range(len(config_list))])

LLM models:  ['gpt-4', 'gpt-4-32k']


It first looks for environment variable "OAI_CONFIG_LIST" which needs to be a valid json string. If that variable is not found, it then looks for a json file named "OAI_CONFIG_LIST". It filters the configs by models (you can filter by other keys as well).

The config list looks like the following:
```python
config_list = [
    {
        "model": "gpt-4",
        "api_key": "<your OpenAI API key>",
    },  # OpenAI API endpoint for gpt-4
    {
        "model": "gpt-35-turbo-0631",  # 0631 or newer is needed to use functions
        "base_url": "<your Azure OpenAI API base>", 
        "api_type": "azure", 
        "api_version": "2023-08-01-preview", # 2023-07-01-preview or newer is needed to use functions
        "api_key": "<your Azure OpenAI API key>"
    }
]
```

You can set the value of config_list in any way you prefer. Please refer to this [notebook](https://github.com/microsoft/autogen/blob/main/notebook/oai_openai_utils.ipynb) for full code examples of the different methods.

## Construct Agents

In [3]:
import chromadb

def chroma_query(path, collection_name, query):
    chroma_client = chromadb.PersistentClient(path=path)
    collection = chroma_client.get_or_create_collection(name=collection_name)
    results = collection.query(query_texts=[query], n_results=2)
    return results

In [4]:
llm_config = {
    "timeout": 60,
    "cache_seed": 42,
    "config_list": config_list,
    "temperature": 0,
}

# autogen.ChatCompletion.start_logging()


def termination_msg(x):
    return isinstance(x, dict) and "TERMINATE" == str(x.get("content", ""))[-9:].upper()



user_proxy = autogen.UserProxyAgent(
    name="Admin",
    system_message="A human admin. Interact with the planner to discuss the plan. Plan execution needs to be approved by this admin.",
    human_input_mode="ALWAYS",
    code_execution_config=False,  # we don't want to execute code in this case.
)

planner = autogen.AssistantAgent(
    name="Planner",
    system_message="""Planner. Suggest a plan. Revise the plan based on feedback from admin and critic, until admin approval.
The plan may involve an Financial_Analyst_Writer who can write financial news summary and a Database_Information_Retriever who can retrieve the latest news articles for give topic from the database, a Content_Review_Scorer who can assess the accuracy and relevance of information shared in the chat, a Summary_Improvement_Advisor who can suggest improvements to the summary, and a Final_Summary_Validator who can validate the final summary.
Explain the plan first. Be clear which step is performed by an Database_Information_Retriever, and which step is performed by a Financial_Analyst_Writer.
""",
    llm_config=llm_config,
)

Financial_Analyst_Writer = AssistantAgent(
    name="Financial_Analyst_Writer",
    llm_config=llm_config,
    is_termination_msg=termination_msg,
    human_input_mode="NEVER",
    system_message= """You are now in a group chat. You need to complete a task with other participants. As a Financial Analyst Writer, your role involves generating concise and informative summaries of news articles or write an article on given financial topics. If you can't answer the question without the current context in group chat, you should reply exactly `UPDATE CONTEXT`. After incorporating the improvements, you will present the updated summary back to the author for feedback. 
In your role, you have the autonomy to question the provided content or the process presented in this group chat and can request corrections or seek clarification if there is something that appears to be missing or unclear after executing a given task. If at any point you find yourself confused or in need of assistance, do not hesitate to reach out to the group chat manager, who can guide you or delegate the task to another qualified participant. Your proactive communication and feedback will ensure the delivery of high-quality, timely financial news summaries to your team.
""",
    code_execution_config=False,  # we don't want to execute code in this case.
    default_auto_reply="Reply `TERMINATE` if the task is done.",
    description="Financial_Analyst_Writer is a role involves generating concise and informative summaries of news articles or write an article on given financial topics.",
)

Content_Review_Scorer = AssistantAgent(
    name="Content_Review_Scorer",
    llm_config=llm_config,
    is_termination_msg=termination_msg,
    human_input_mode="NEVER",
    system_message="""You are now in a group chat where your primary role as a Content_Review_Scorer is to generate a summary of financial news articles, review the produced summaries, score them, and provide feedback for improvement. As a financial analyst preparing crucial summaries for your colleagues, you must ensure accuracy and comprehensiveness in the conveyed information. Your task involves:
- Carefully reading and understanding the thematic content of news articles in the financial sector.
- Reviewing and critically assessing the quality of these summaries, assigning an appropriate score based on predefined criteria such as relevance, accuracy, completeness, and clarity.
- Offering constructive feedback aimed at enhancing the summarizer's work, highlighting areas of the summary that may require additional information or could benefit from refinement.
- When necessary, retrieving missing information from a database to supplement and improve the summary's content, ensuring that all relevant details are included.
- After the summarizer has integrated your feedback and revised the summary, performing a final review to confirm that all improvements have been effectively made.
- Providing a final score for the improved summary, reflecting any enhancements made as a result of your feedback.
Your engagement in this task is iterative, and you will depend on your expertise in financial analysis to make judgements about the content. If at any point you are unsure or need clarification, you are encouraged to seek assistance from the group chat manager, who will facilitate further discussion or select another participant to provide support.
Ultimately, once you are satisfied that the summary is of high quality and meets the requirements, at which point you should reply `TERMINATE` to signal the end of the task.
This role does not involve coding; instead, you will be leveraging your financial expertise and communication skills to accomplish these tasks. If you encounter any issues, such as a lack of output after an information retrieval attempt, feel free to voice your concerns and work collaboratively to correct any oversights.
""",
    code_execution_config=False,  # we don't want to execute code in this case.
    default_auto_reply="Reply `TERMINATE` if the task is done.",
    description="The Content_Review_Scorer with main task is to review and score summaries of financial news articles, ensuring they are accurate, comprehensive, and clear.", 
    )

# Summary_Improvement_Advisor = AssistantAgent(
#     name="Summary_Improvement_Advisor",
#     llm_config=llm_config,
#     is_termination_msg=termination_msg,
#     human_input_mode="NEVER",
#     system_message="You are now in a group chat tasked with the critical role of Summary_Improvement_Advisor for financial analysts. As an advanced AI assistant, you possess the expertise to assess and enhance financial summaries using your language skills. Although python coding isn't within your scope, you should be capable of extracting pertinent information from various sources without the need for scripts.\n\nYour role involves meticulously reviewing summaries created by a financial analyst, assigning them a score based on accuracy, coherence, and completeness, and offering constructive feedback. Whenever a summary lacks certain details or could be improved, you will retrieve the necessary information, likely from financial databases, and refine the content aspiring for excellence.\n\nWhen facing challenges or if the task appears to be at a standstill, do not hesitate to seek guidance from the group chat manager, who has the authority to appoint another participant to assist or to take over the task as needed. Keep in mind the collaborative nature of this environment; you have the autonomy to question any previous messages or work presented in the group chat and suggest appropriate amendments.\n\nYour objective is to work with financial analysts to ensure the final summary is of superior quality, reflecting the most significant aspects of the financial topics at hand. The overall success is marked by both the analyst's satisfaction with your improvements and the final score you attribute to the summary.\n\nWhen you believe that the task has been satisfactorily completed—that is, the final summary is polished, and all feedback has been addressed—you should conclude the interaction with `TERMINATE`.",
#     code_execution_config=False,  # we don't want to execute code in this case.
#     default_auto_reply="Reply `TERMINATE` if the task is done.",
#     description="The Summary_Improvement_Advisor is a critical thinker with strong analytical skills and expertise in Python, capable of scrutinizing previous discussions or Python code snippets shared in the group chat. This individual must excel in clear communication to provide constructive feedback and alternative solutions that enhance the group's understanding and productivity. They should be encouraged to speak up whenever they spot inaccuracies or areas for improvement to ensure optimal group outcomes.",
# )

# Final_Summary_Validator = AssistantAgent(
#     name="Final_Summary_Validator",
#     llm_config=llm_config,
#     is_termination_msg=termination_msg,
#     human_input_mode="NEVER",
#     system_message="You are now in a group chat. You need to complete a task with other participants. As a Final_Summary_Validator, your role is to critically evaluate and enhance summaries of financial news articles created by a financial analyst for an internal audience. \n\nYour task involves:\n\n1. Generating a summary of the news articles on a given financial topic using your analytical skill.\n2. Reviewing this summary carefully, assigning it a score based on its accuracy, coverage, and readability, and providing constructive feedback to the summarizer.\n3. Retrieving any missing or additional pertinent information from a database to refine the summary without using code. You are expected to have the necessary access and authorization to handle relevant data securely.\n4. Improving the summary based on the retrieved information and re-evaluating it, ensuring that the final product is comprehensive and up to the expected standard.\n5. Providing the author with a re-scored and revised summary and awaiting their feedback.\n6. Once the author's feedback is received, you will provide a final summary and its final score.\n\nRemember, your role does not involve coding. Should you encounter difficulties or uncertainties while executing your tasks, do not hesitate to ask for help from the group chat manager. They will guide you or redirect you to another team member. \n\nIn case of doubts about previous messages, such as a lack of output after an analyst has claimed to improve the summary, you should voice your concerns and contribute a corrected approach.\n\nEnd the task by stating `TERMINATE` in the group chat when you have determined that the task requirements have been fully satisfied.",
#     code_execution_config=False,  # we don't want to execute code in this case.
#     default_auto_reply="Reply `TERMINATE` if the task is done.",
#     description="Final_Summary_Validator is a critical thinker and detail-oriented individual who possesses strong analytical skills and a good understanding of Python coding. This role is responsible for scrutinizing previous messages and code within the group chat, identifying any inaccuracies or errors, and providing a corrected summary or revised Python code when necessary. The manager should allow the Final_Summary_Validator to interject when clarity and accuracy are paramount, especially following discussions that have led to potentially flawed conclusions or code without expected outputs.",
# )

Database_Information_Retriever = RetrieveUserProxyAgent(
    name="Database_Information_Retriever",
    # system_message="You are now in a group chat where your primary role as a Content_Review_Scorer is to generate a summary of financial news articles, review the produced summaries, score them, and provide feedback for improvement. As a financial analyst preparing crucial summaries for your colleagues, you must ensure accuracy and comprehensiveness in the conveyed information. Your task involves:\n\n- Carefully reading and understanding the thematic content of news articles in the financial sector.\n- Creating clear and concise summaries that capture the key points and essential data.\n- Reviewing and critically assessing the quality of these summaries, assigning an appropriate score based on predefined criteria such as relevance, accuracy, completeness, and clarity.\n- Offering constructive feedback aimed at enhancing the summarizer's work, highlighting areas of the summary that may require additional information or could benefit from refinement.\n- When necessary, retrieving missing information from a database to supplement and improve the summary's content, ensuring that all relevant details are included.\n- After the summarizer has integrated your feedback and revised the summary, performing a final review to confirm that all improvements have been effectively made.\n- Providing a final score for the improved summary, reflecting any enhancements made as a result of your feedback.\n\nYour engagement in this task is iterative, and you will depend on your expertise in financial analysis to make judgements about the content. If at any point you are unsure or need clarification, you are encouraged to seek assistance from the group chat manager, who will facilitate further discussion or select another participant to provide support.\n\nUltimately, once you are satisfied that the summary is of high quality and meets the requirements, and once all your colleagues in the chat are content with the resolved issues, you or another participant may conclude the session with `TERMINATE.` \n\nThis role does not involve coding; instead, you will be leveraging your financial expertise and communication skills to accomplish these tasks. If you encounter any issues, such as a lack of output after an information retrieval attempt, feel free to voice your concerns and work collaboratively to correct any oversights.",
    is_termination_msg=termination_msg,
    human_input_mode="NEVER",
    max_consecutive_auto_reply=3,
    # llm_config=llm_config,
    retrieve_config={
        "task": "qa",
        "docs_path": None,
        "model": config_list[0]["model"],
        "client": chromadb.PersistentClient(path="/workspaces/autogen/usecases/nb/chroma_reuters21578"),
        "collection_name": "test",
        "get_or_create": True,
    },
    code_execution_config=False,  # we don't want to execute code in this case.
    description="Database_Information_Retriever is a role that involves retrieving information from a database",
)


In [7]:
# PROBLEM = "summarized news about Agip Petroli"
PROBLEM = """summarized news about:
How are the Federal Reserve (Fed) and the European Central Bank (ECB) currently adjusting interest rates, and what are their reasons?
What quantitative easing measures have the Fed and the ECB introduced or terminated recently?
How have the Fed and the ECB's policies affected the inflation rates in the U.S. and Eurozone, respectively?
What are the latest updates on the Fed and the ECB's outlooks on economic growth?
How are emerging market central banks responding to the monetary policies set by the Fed and the ECB?
Have the Fed or the ECB made any statements regarding concerns over a potential recession?
What are the latest labor market conditions in the U.S. and Eurozone, and how are they influencing the Fed and ECB's policy decisions?
Have there been any recent changes in the Fed and ECB's stance on fiscal stimulus measures?
How are the Fed and the ECB addressing concerns about asset bubbles or financial stability?
What measures are central banks taking to mitigate risks associated with high government debt levels?
How are central banks, including the Fed and the ECB, planning to handle potential deflationary pressures?
What role are the Fed and the ECB playing in addressing currency volatility in the forex markets?
Have there been any significant shifts in the Fed and ECB's currency reserve policies?
What are the implications of the current monetary policies of the Fed and ECB on global commodity prices?
Are there any new regulatory changes or supervisory actions that the Fed and ECB have announced which could impact banking and financial services?
"""

# PROBLEM = [
#     "How are the Federal Reserve (Fed) and the European Central Bank (ECB) currently adjusting interest rates, and what are their reasons?",
#     "What quantitative easing measures have the Fed and the ECB introduced or terminated recently?",
#     "How have the Fed and the ECB's policies affected the inflation rates in the U.S. and Eurozone, respectively?",
#     "What are the latest updates on the Fed and the ECB's outlooks on economic growth?",
#     "How are emerging market central banks responding to the monetary policies set by the Fed and the ECB?",
#     "Have the Fed or the ECB made any statements regarding concerns over a potential recession?",
#     "What are the latest labor market conditions in the U.S. and Eurozone, and how are they influencing the Fed and ECB's policy decisions?",
#     "Have there been any recent changes in the Fed and ECB's stance on fiscal stimulus measures?",
#     "How are the Fed and the ECB addressing concerns about asset bubbles or financial stability?",
# ]

def _reset_agents():
    # user_proxy.reset()
    Financial_Analyst_Writer.reset()
    Content_Review_Scorer.reset()
    # Summary_Improvement_Advisor.reset()
    # Final_Summary_Validator.reset()
    Database_Information_Retriever.reset()



_reset_agents()
groupchat = autogen.GroupChat(
    agents=[Database_Information_Retriever, Financial_Analyst_Writer, Content_Review_Scorer],
    messages=[], max_round=12, speaker_selection_method="auto"
)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

In [8]:
client = chromadb.PersistentClient(path="/workspaces/autogen/usecases/nb/chroma_reuters21578")
client.list_collections()

[Collection(name=test)]

In [9]:
# Start chatting with boss_aid as this is the user proxy agent.
Database_Information_Retriever.initiate_chat(
    manager,
    message=Database_Information_Retriever.message_generator,
    problem=PROBLEM,
    n_results=3,
)

doc_ids:  [['1357', '3054', '243']]
Adding doc_id 1357 to context.
Adding doc_id 3054 to context.
Adding doc_id 243 to context.
Database_Information_Retriever (to chat_manager):

You're a retrieve augmented chatbot. You answer user's questions based on your own knowledge and the
context provided by the user.
If you can't answer the question with or without the current context, you should reply exactly `UPDATE CONTEXT`.
You must give as short an answer as possible.

User's question is: summarized news about:How are the Federal Reserve (Fed) and the European Central Bank (ECB) currently adjusting interest rates, and what are their reasons?
What quantitative easing measures have the Fed and the ECB introduced or terminated recently?
How have the Fed and the ECB's policies affected the inflation rates in the U.S. and Eurozone, respectively?
What are the latest updates on the Fed and the ECB's outlooks on economic growth?
How are emerging market central banks responding to the monetary poli

ChatResult(chat_id=None, chat_history=[{'content': 'You\'re a retrieve augmented chatbot. You answer user\'s questions based on your own knowledge and the\ncontext provided by the user.\nIf you can\'t answer the question with or without the current context, you should reply exactly `UPDATE CONTEXT`.\nYou must give as short an answer as possible.\n\nUser\'s question is: summarized news about:How are the Federal Reserve (Fed) and the European Central Bank (ECB) currently adjusting interest rates, and what are their reasons?\nWhat quantitative easing measures have the Fed and the ECB introduced or terminated recently?\nHow have the Fed and the ECB\'s policies affected the inflation rates in the U.S. and Eurozone, respectively?\nWhat are the latest updates on the Fed and the ECB\'s outlooks on economic growth?\nHow are emerging market central banks responding to the monetary policies set by the Fed and the ECB?\nHave the Fed or the ECB made any statements regarding concerns over a potentia

In [10]:
_reset_agents()

PROBLEM = "summarized news about Agip Petroli"

# Start chatting with boss_aid as this is the user proxy agent.
Database_Information_Retriever.initiate_chat(
    manager,
    message=Database_Information_Retriever.message_generator,
    problem=PROBLEM,
    n_results=3,
)

doc_ids:  [['1677', '189', '398']]
Adding doc_id 1677 to context.
Adding doc_id 189 to context.
Adding doc_id 398 to context.
Database_Information_Retriever (to chat_manager):

You're a retrieve augmented chatbot. You answer user's questions based on your own knowledge and the
context provided by the user.
If you can't answer the question with or without the current context, you should reply exactly `UPDATE CONTEXT`.
You must give as short an answer as possible.

User's question is: summarized news about Agip Petroli

Context is: State oil firm Agip Petroli Spa said it has
acquired a 50 pct stake in Steuart Petroleum Co, an independent
U.S. Oil products company. Financial terms were not disclosed.
    Agip Petroli, a subsidiary of state energy concern Ente
Nazionali Idrocarburi, said in a statement that the remaining
50 pct of the U.S. Firm is owned by Steuart Investment Co, a
holding company that also operates in the sectors of
transportation, hotels and insurance.
    The Italian fir

Financial_Analyst_Writer (to chat_manager):

Agip Petroli Spa acquired a 50% stake in Steuart Petroleum Co, a U.S. oil products company. Financial terms were not disclosed. The remaining 50% is owned by Steuart Investment Co. Steuart Petroleum operates primarily on the East Coast.

--------------------------------------------------------------------------------
Content_Review_Scorer (to chat_manager):

Relevance: 5/5
Accuracy: 5/5
Completeness: 4/5
Clarity: 5/5

Feedback: The summary captures the main points of the news article accurately and clearly. However, it could be improved by including the information that Agip Petroli is a subsidiary of Ente Nazionali Idrocarburi, which adds context to the acquisition. Additionally, mentioning that Steuart Investment Co is involved in other sectors could provide a more comprehensive view of the company's profile.

--------------------------------------------------------------------------------
Database_Information_Retriever (to chat_manager):


ChatResult(chat_id=None, chat_history=[{'content': "You're a retrieve augmented chatbot. You answer user's questions based on your own knowledge and the\ncontext provided by the user.\nIf you can't answer the question with or without the current context, you should reply exactly `UPDATE CONTEXT`.\nYou must give as short an answer as possible.\n\nUser's question is: summarized news about Agip Petroli\n\nContext is: State oil firm Agip Petroli Spa said it has\nacquired a 50 pct stake in Steuart Petroleum Co, an independent\nU.S. Oil products company. Financial terms were not disclosed.\n    Agip Petroli, a subsidiary of state energy concern Ente\nNazionali Idrocarburi, said in a statement that the remaining\n50 pct of the U.S. Firm is owned by Steuart Investment Co, a\nholding company that also operates in the sectors of\ntransportation, hotels and insurance.\n    The Italian firm said Steuart Petroleum operates primarily\non the East Coast.\n REUTER\n\n&lt;Montedison SPA> of Italy said 